In [1]:
import findspark
findspark.init()
import pyspark
import random

import sys
from operator import add

from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark import SparkConf

import pyspark
import random

#1 Set up spark context and SparkSession
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [2]:
# create dataframe
spark_df1 = spark.createDataFrame([
            ('John', 32),
            ('David', 22),
            ('Raj', 10),
            ('Andy', 25)],
            ["name", "age"])

spark_df1.registerTempTable("test_df")


# generate checksum column SHA-2 family of hash functions 256 bits
from pyspark.sql.functions import sha2, concat_ws, md5
spark_df1.withColumn("row_sha2", sha2(concat_ws("||", *spark_df1.columns), 256)).show(truncate=False)
history_data_df = spark_df1.withColumn("row_sha2", sha2(concat_ws("||", *spark_df1.columns), 256))
# changed dataframe 
spark_df2 = spark.createDataFrame([
            ('John', 33),
            ('David', 22),
            ('Raj', 11),
#            ('Andy', 25),
            ('Paul', 28)],
            ["name", "age"])


col_list_to_hash = ['name','age']
# encoding based on column list
spark_df2.withColumn("row_sha2", sha2(concat_ws("||", *col_list_to_hash), 256)).show(truncate=False)
delta_df = spark_df2.withColumn("row_sha2", sha2(concat_ws("||", *col_list_to_hash), 256))
# encoding based on all columns
spark_df2.withColumn("row_sha2", sha2(concat_ws("||", *spark_df1.columns), 256)).show(truncate=False)

# md5 encoding
spark_df2.withColumn("row_md5", md5(concat_ws("||", *col_list_to_hash))).show(truncate=False)


+-----+---+----------------------------------------------------------------+
|name |age|row_sha2                                                        |
+-----+---+----------------------------------------------------------------+
|John |32 |afabe3fe5711744f1a21433e21956111543727cae2f96e1a83bad2836da3c0ce|
|David|22 |eb17e164bc6dd6c0521278fa34aeb0da179acfd247192d2f8c5d948518c8f2b7|
|Raj  |10 |c37288dc8fe43e4f30f55ac9a93013555ffc1e9146acfb94416cb9f72d051de1|
|Andy |25 |737fc02c0333519807e095c2d753d348a09063a20a9ede1c2da2e784e23551d5|
+-----+---+----------------------------------------------------------------+

+-----+---+----------------------------------------------------------------+
|name |age|row_sha2                                                        |
+-----+---+----------------------------------------------------------------+
|John |33 |d486ab8a3b8333511bd12758895b6f1ae532e1d599f8acbe438d7d5ca4aebd3e|
|David|22 |eb17e164bc6dd6c0521278fa34aeb0da179acfd247192d2f8c5d948518c8f2b7

In [3]:
# spark_df1 is history and spark_df2 delta based on keyCols
keyCols = ['name']
New_updated_records_replaceDf = spark_df1.alias('a').join(spark_df2.alias('b'), on=keyCols, how='inner').select('a.*')
New_updated_records_replaceDf.show()
resultDf = spark_df1.subtract(New_updated_records_replaceDf).union(spark_df2).show()

+-----+---+
| name|age|
+-----+---+
| John| 32|
|  Raj| 10|
|David| 22|
+-----+---+

+-----+---+
| name|age|
+-----+---+
| Andy| 25|
| John| 33|
|David| 22|
|  Raj| 11|
| Paul| 28|
+-----+---+



In [4]:
# spark_df1 is history and spark_df2 delta based on keyCols
keyCols = ['name','age']
New_updated_records_replaceDf = history_data_df.alias('a').join(delta_df.alias('b'), on=keyCols, how='inner').select('a.*')
New_updated_records_replaceDf.show()
resultDf = history_data_df.subtract(New_updated_records_replaceDf).union(delta_df).show()

+-----+---+--------------------+
| name|age|            row_sha2|
+-----+---+--------------------+
|David| 22|eb17e164bc6dd6c05...|
+-----+---+--------------------+

+-----+---+--------------------+
| name|age|            row_sha2|
+-----+---+--------------------+
| Andy| 25|737fc02c033351980...|
| John| 32|afabe3fe5711744f1...|
|  Raj| 10|c37288dc8fe43e4f3...|
| John| 33|d486ab8a3b8333511...|
|David| 22|eb17e164bc6dd6c05...|
|  Raj| 11|d0f22ed3e129a7b75...|
| Paul| 28|b71b0817d3973d4b1...|
+-----+---+--------------------+



In [5]:
old_entries_to_be_deleted = history_data_df.subtract(delta_df)
old_entries_to_be_deleted.show()

+----+---+--------------------+
|name|age|            row_sha2|
+----+---+--------------------+
|Andy| 25|737fc02c033351980...|
|John| 32|afabe3fe5711744f1...|
| Raj| 10|c37288dc8fe43e4f3...|
+----+---+--------------------+



In [6]:
new_entries_to_be_appened = delta_df.subtract(history_data_df)
new_entries_to_be_appened.show()

+----+---+--------------------+
|name|age|            row_sha2|
+----+---+--------------------+
| Raj| 11|d0f22ed3e129a7b75...|
|John| 33|d486ab8a3b8333511...|
|Paul| 28|b71b0817d3973d4b1...|
+----+---+--------------------+



In [7]:
# new keys, so these will be inserted
keyCols = ['name']
new_entries_to_be_inserted = delta_df[keyCols].subtract(history_data_df[keyCols])
new_entries_to_be_inserted.join(delta_df, on=keyCols, how='left').show()


+----+---+--------------------+
|name|age|            row_sha2|
+----+---+--------------------+
|Paul| 28|b71b0817d3973d4b1...|
+----+---+--------------------+



In [8]:
# existing keys
keyCols = ['name']
#changed_entries_to_be_updated = delta_df[keyCols].alias('a').join(history_data_df[keyCols], on=keyCols, how='inner').select('a.*')
changed_entries_to_be_updated = delta_df.alias('a').join(history_data_df, on=keyCols, how='inner')
changed_entries_to_be_updated.show()

+-----+---+--------------------+---+--------------------+
| name|age|            row_sha2|age|            row_sha2|
+-----+---+--------------------+---+--------------------+
| John| 33|d486ab8a3b8333511...| 32|afabe3fe5711744f1...|
|  Raj| 11|d0f22ed3e129a7b75...| 10|c37288dc8fe43e4f3...|
|David| 22|eb17e164bc6dd6c05...| 22|eb17e164bc6dd6c05...|
+-----+---+--------------------+---+--------------------+



In [9]:
# existing keys where records are changed comparing based on hash key
delta_df.alias('a').join(history_data_df, on=keyCols, how='inner').filter(history_data_df.row_sha2 != delta_df.row_sha2).show()

+----+---+--------------------+---+--------------------+
|name|age|            row_sha2|age|            row_sha2|
+----+---+--------------------+---+--------------------+
|John| 33|d486ab8a3b8333511...| 32|afabe3fe5711744f1...|
| Raj| 11|d0f22ed3e129a7b75...| 10|c37288dc8fe43e4f3...|
+----+---+--------------------+---+--------------------+



In [10]:
#Simulating an UPSERT between 2 datasets using pySpark

In [11]:
# existing data
current_data_df1 = spark.createDataFrame([('1','John','30','1000','01/08/2019  12:00:00'),
('2','Peter','35','1500','02/08/2019  12:00:00'),
('3','Gabe','21','800','03/08/2019  12:00:00'),
('4','Oscar','29','2000','04/08/2019  12:00:00'),
('5','Anna','20','1200','05/08/2019  12:00:00')],
                             ['id','name','age','salary','last_modified_date'])

increamental_data_df1 = spark.createDataFrame([('1','John','43','3000','12/08/2019  12:00:00'),
('2','Peter','35','3500','12/08/2019  12:00:00'),
('3','Gabe','21','800','12/08/2019  12:00:00'),
('6','Patricia','40','2500','12/08/2019  12:00:00')],
                             ['id','name','age','salary','last_modified_date'])


In [12]:
from datetime import datetime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType

In [13]:
# convert string format to datetime format
from pyspark.sql.functions import unix_timestamp, from_unixtime
from pyspark.sql.functions import expr, col
# faster if else condition in pyspark
from pyspark.sql import functions as F

current_data_df1 = current_data_df1.withColumn("new_last_modified_date",expr("to_timestamp(last_modified_date, 'dd/MM/yyyy  HH:mm:ss')"))
increamental_data_df1 = increamental_data_df1.withColumn("new_last_modified_date",expr("to_timestamp(last_modified_date, 'dd/MM/yyyy  HH:mm:ss')"))
current_data_df1 = current_data_df1.withColumn("record_load_time",F.col('new_last_modified_date'))
increamental_data_df1 = increamental_data_df1.withColumn("record_load_time",F.col('new_last_modified_date'))

current_data_df1.show()
increamental_data_df1.show()

+---+-----+---+------+--------------------+----------------------+-------------------+
| id| name|age|salary|  last_modified_date|new_last_modified_date|   record_load_time|
+---+-----+---+------+--------------------+----------------------+-------------------+
|  1| John| 30|  1000|01/08/2019  12:00:00|   2019-08-01 12:00:00|2019-08-01 12:00:00|
|  2|Peter| 35|  1500|02/08/2019  12:00:00|   2019-08-02 12:00:00|2019-08-02 12:00:00|
|  3| Gabe| 21|   800|03/08/2019  12:00:00|   2019-08-03 12:00:00|2019-08-03 12:00:00|
|  4|Oscar| 29|  2000|04/08/2019  12:00:00|   2019-08-04 12:00:00|2019-08-04 12:00:00|
|  5| Anna| 20|  1200|05/08/2019  12:00:00|   2019-08-05 12:00:00|2019-08-05 12:00:00|
+---+-----+---+------+--------------------+----------------------+-------------------+

+---+--------+---+------+--------------------+----------------------+-------------------+
| id|    name|age|salary|  last_modified_date|new_last_modified_date|   record_load_time|
+---+--------+---+------+-----------

In [14]:
# one more way convert string into date field
from pyspark.sql.functions import to_utc_timestamp, unix_timestamp, lit, datediff, col
df_test1 = spark.createDataFrame([('1','John','30','1000','1/08/2019  12:00:00'),
('2','Peter','35','1500','2/08/2019  12:00:00'),
('3','Gabe','21','800','3/08/2019  12:00:00'),
('4','Oscar','29','2000','14/08/2019  12:00:00'),
('5','Anna','20','1200','5/08/2019  12:00:00')],
                             ['id','name','age','salary','last_modified_date'])

timeFmt = 'dd/MM/yyyy  HH:mm:ss'
df_test1 = df_test1.withColumn('NewInvoiceDate'
                 , to_utc_timestamp(unix_timestamp(col('last_modified_date'),timeFmt).cast('timestamp')
                 , 'UTC'))
df_test1.show()

from pyspark.sql.functions import col
print('Before type changes \n', df_test1.dtypes)
# cast different columns data types to integer, double, string
for c in ['id','age']:
    df_test1=df_test1.withColumn(c, df_test1[c].cast('integer'))
for c in ['salary']:
    df_test1=df_test1.withColumn(c, df_test1[c].cast('double'))
for c in ['name']:
    df_test1=df_test1.withColumn(c, df_test1[c].cast('string'))

print('After type changes \n',df_test1.dtypes)

+---+-----+---+------+--------------------+-------------------+
| id| name|age|salary|  last_modified_date|     NewInvoiceDate|
+---+-----+---+------+--------------------+-------------------+
|  1| John| 30|  1000| 1/08/2019  12:00:00|2019-08-01 12:00:00|
|  2|Peter| 35|  1500| 2/08/2019  12:00:00|2019-08-02 12:00:00|
|  3| Gabe| 21|   800| 3/08/2019  12:00:00|2019-08-03 12:00:00|
|  4|Oscar| 29|  2000|14/08/2019  12:00:00|2019-08-14 12:00:00|
|  5| Anna| 20|  1200| 5/08/2019  12:00:00|2019-08-05 12:00:00|
+---+-----+---+------+--------------------+-------------------+

Before type changes 
 [('id', 'string'), ('name', 'string'), ('age', 'string'), ('salary', 'string'), ('last_modified_date', 'string'), ('NewInvoiceDate', 'timestamp')]
After type changes 
 [('id', 'int'), ('name', 'string'), ('age', 'int'), ('salary', 'double'), ('last_modified_date', 'string'), ('NewInvoiceDate', 'timestamp')]


In [15]:
# Both DataFrames are grouped together with union (which is equivalent to UNION ALL in SQL), creating the 3rd and final DataFrame. 
df_upsert = current_data_df1.union(increamental_data_df1)
df_upsert.show(truncate=False)

+---+--------+---+------+--------------------+----------------------+-------------------+
|id |name    |age|salary|last_modified_date  |new_last_modified_date|record_load_time   |
+---+--------+---+------+--------------------+----------------------+-------------------+
|1  |John    |30 |1000  |01/08/2019  12:00:00|2019-08-01 12:00:00   |2019-08-01 12:00:00|
|2  |Peter   |35 |1500  |02/08/2019  12:00:00|2019-08-02 12:00:00   |2019-08-02 12:00:00|
|3  |Gabe    |21 |800   |03/08/2019  12:00:00|2019-08-03 12:00:00   |2019-08-03 12:00:00|
|4  |Oscar   |29 |2000  |04/08/2019  12:00:00|2019-08-04 12:00:00   |2019-08-04 12:00:00|
|5  |Anna    |20 |1200  |05/08/2019  12:00:00|2019-08-05 12:00:00   |2019-08-05 12:00:00|
|1  |John    |43 |3000  |12/08/2019  12:00:00|2019-08-12 12:00:00   |2019-08-12 12:00:00|
|2  |Peter   |35 |3500  |12/08/2019  12:00:00|2019-08-12 12:00:00   |2019-08-12 12:00:00|
|3  |Gabe    |21 |800   |12/08/2019  12:00:00|2019-08-12 12:00:00   |2019-08-12 12:00:00|
|6  |Patri

In [16]:
from pyspark.sql.window import Window
import pyspark.sql.functions as func
from pyspark.sql.functions import *
from pyspark.sql.functions import desc, asc
# Set primary keys of dataframe
# It could also be a composite primary key, hence the list (e.g ['id1','id2','id3'])
#primary_keys = ['id','name','age']
primary_keys = ['id']

# Partition dataset by primary key and order by the date field
# This step will group duplicated IDs together, putting the latest (date) on top
w = Window.partitionBy(*primary_keys).orderBy(desc('new_last_modified_date'),asc('id'))


In [17]:
#Now, a temporary column ‘_row_number’ is created using the window created above. Note how the DataFrame looks like, with the primary key ‘id’ grouped together and ordered by ‘last_modified_date’ (descending):

df_upsert = df_upsert.withColumn("_row_number", row_number().over(w))

df_upsert.filter("id <= 10").sort(col("id").asc()).show(truncate=False)

+---+--------+---+------+--------------------+----------------------+-------------------+-----------+
|id |name    |age|salary|last_modified_date  |new_last_modified_date|record_load_time   |_row_number|
+---+--------+---+------+--------------------+----------------------+-------------------+-----------+
|1  |John    |43 |3000  |12/08/2019  12:00:00|2019-08-12 12:00:00   |2019-08-12 12:00:00|1          |
|1  |John    |30 |1000  |01/08/2019  12:00:00|2019-08-01 12:00:00   |2019-08-01 12:00:00|2          |
|2  |Peter   |35 |3500  |12/08/2019  12:00:00|2019-08-12 12:00:00   |2019-08-12 12:00:00|1          |
|2  |Peter   |35 |1500  |02/08/2019  12:00:00|2019-08-02 12:00:00   |2019-08-02 12:00:00|2          |
|3  |Gabe    |21 |800   |12/08/2019  12:00:00|2019-08-12 12:00:00   |2019-08-12 12:00:00|1          |
|3  |Gabe    |21 |800   |03/08/2019  12:00:00|2019-08-03 12:00:00   |2019-08-03 12:00:00|2          |
|4  |Oscar   |29 |2000  |04/08/2019  12:00:00|2019-08-04 12:00:00   |2019-08-04 12

In [18]:
from itertools import chain
from pyspark.sql.functions import create_map, lit

recordTypeDict = {1:'#keep this new updated record',2:'#remove this record'}

mapping_expr = create_map([lit(x) for x in chain(*recordTypeDict.items())])

df_upsert = df_upsert.withColumn('record_type', mapping_expr[df_upsert['_row_number']])
df_upsert.filter("id <= 10").sort(col("id").asc()).show()

+---+--------+---+------+--------------------+----------------------+-------------------+-----------+--------------------+
| id|    name|age|salary|  last_modified_date|new_last_modified_date|   record_load_time|_row_number|         record_type|
+---+--------+---+------+--------------------+----------------------+-------------------+-----------+--------------------+
|  1|    John| 43|  3000|12/08/2019  12:00:00|   2019-08-12 12:00:00|2019-08-12 12:00:00|          1|#keep this new up...|
|  1|    John| 30|  1000|01/08/2019  12:00:00|   2019-08-01 12:00:00|2019-08-01 12:00:00|          2| #remove this record|
|  2|   Peter| 35|  3500|12/08/2019  12:00:00|   2019-08-12 12:00:00|2019-08-12 12:00:00|          1|#keep this new up...|
|  2|   Peter| 35|  1500|02/08/2019  12:00:00|   2019-08-02 12:00:00|2019-08-02 12:00:00|          2| #remove this record|
|  3|    Gabe| 21|   800|03/08/2019  12:00:00|   2019-08-03 12:00:00|2019-08-03 12:00:00|          2| #remove this record|
|  3|    Gabe| 2

In [19]:
# these delta records will be appened to current target table
df_to_append = df_upsert.where(df_upsert._row_number == 1).drop("_row_number")
df_to_append.orderBy('id').show()

+---+--------+---+------+--------------------+----------------------+-------------------+--------------------+
| id|    name|age|salary|  last_modified_date|new_last_modified_date|   record_load_time|         record_type|
+---+--------+---+------+--------------------+----------------------+-------------------+--------------------+
|  1|    John| 43|  3000|12/08/2019  12:00:00|   2019-08-12 12:00:00|2019-08-12 12:00:00|#keep this new up...|
|  2|   Peter| 35|  3500|12/08/2019  12:00:00|   2019-08-12 12:00:00|2019-08-12 12:00:00|#keep this new up...|
|  3|    Gabe| 21|   800|12/08/2019  12:00:00|   2019-08-12 12:00:00|2019-08-12 12:00:00|#keep this new up...|
|  4|   Oscar| 29|  2000|04/08/2019  12:00:00|   2019-08-04 12:00:00|2019-08-04 12:00:00|#keep this new up...|
|  5|    Anna| 20|  1200|05/08/2019  12:00:00|   2019-08-05 12:00:00|2019-08-05 12:00:00|#keep this new up...|
|  6|Patricia| 40|  2500|12/08/2019  12:00:00|   2019-08-12 12:00:00|2019-08-12 12:00:00|#keep this new up...|
+

In [20]:
# these records will be reversed to delete from current target table
df_to_reverse = df_upsert.where(df_upsert._row_number > 1).drop("_row_number")
df_to_reverse.orderBy('id').show()

+---+-----+---+------+--------------------+----------------------+-------------------+-------------------+
| id| name|age|salary|  last_modified_date|new_last_modified_date|   record_load_time|        record_type|
+---+-----+---+------+--------------------+----------------------+-------------------+-------------------+
|  1| John| 30|  1000|01/08/2019  12:00:00|   2019-08-01 12:00:00|2019-08-01 12:00:00|#remove this record|
|  2|Peter| 35|  1500|02/08/2019  12:00:00|   2019-08-02 12:00:00|2019-08-02 12:00:00|#remove this record|
|  3| Gabe| 21|   800|03/08/2019  12:00:00|   2019-08-03 12:00:00|2019-08-03 12:00:00|#remove this record|
+---+-----+---+------+--------------------+----------------------+-------------------+-------------------+



In [21]:
import pyspark.sql.functions as F
# key figures need to be reverse
keyfigures = ['age', 'salary']
# column list except key figures
column_minus_keyfigures_list = [c for c in df_to_reverse.columns if c not in keyfigures]

df_to_reverse_final = df_to_reverse.select( *column_minus_keyfigures_list , *[(F.col(col_name)* -1).alias(col_name) for col_name in keyfigures])
df_to_reverse_final.show()


+---+-----+--------------------+----------------------+-------------------+-------------------+-----+-------+
| id| name|  last_modified_date|new_last_modified_date|   record_load_time|        record_type|  age| salary|
+---+-----+--------------------+----------------------+-------------------+-------------------+-----+-------+
|  3| Gabe|03/08/2019  12:00:00|   2019-08-03 12:00:00|2019-08-03 12:00:00|#remove this record|-21.0| -800.0|
|  1| John|01/08/2019  12:00:00|   2019-08-01 12:00:00|2019-08-01 12:00:00|#remove this record|-30.0|-1000.0|
|  2|Peter|02/08/2019  12:00:00|   2019-08-02 12:00:00|2019-08-02 12:00:00|#remove this record|-35.0|-1500.0|
+---+-----+--------------------+----------------------+-------------------+-------------------+-----+-------+



In [22]:
# with column list keyfigures example 
df_to_reverse.select([c for c in df_to_reverse.columns if c in keyfigures]).show()

# without column list keyfigures example 
df_to_reverse.select([c for c in df_to_reverse.columns if c not in keyfigures]).show()

+---+------+
|age|salary|
+---+------+
| 21|   800|
| 30|  1000|
| 35|  1500|
+---+------+

+---+-----+--------------------+----------------------+-------------------+-------------------+
| id| name|  last_modified_date|new_last_modified_date|   record_load_time|        record_type|
+---+-----+--------------------+----------------------+-------------------+-------------------+
|  3| Gabe|03/08/2019  12:00:00|   2019-08-03 12:00:00|2019-08-03 12:00:00|#remove this record|
|  1| John|01/08/2019  12:00:00|   2019-08-01 12:00:00|2019-08-01 12:00:00|#remove this record|
|  2|Peter|02/08/2019  12:00:00|   2019-08-02 12:00:00|2019-08-02 12:00:00|#remove this record|
+---+-----+--------------------+----------------------+-------------------+-------------------+



In [23]:
# Both DataFrames are grouped together with union (which is equivalent to UNION ALL in SQL), creating the 3rd and final DataFrame with update timestamp. 
# select same column list with same order to union them 
import time
import datetime
from pyspark.sql.functions import lit,unix_timestamp
reverse_timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
df_to_reverse_final = df_to_reverse_final.withColumn('record_load_time',unix_timestamp(lit(reverse_timestamp),'yyyy-MM-dd HH:mm:ss').cast("timestamp"))

append_timestamp = datetime.datetime.fromtimestamp(time.time() + 1).strftime('%Y-%m-%d %H:%M:%S')
df_to_append = df_to_append.withColumn('record_load_time',unix_timestamp(lit(append_timestamp),'yyyy-MM-dd HH:mm:ss').cast("timestamp"))

final_update_df = df_to_reverse_final.union(df_to_append.select(df_to_reverse_final.columns))
final_update_df.show(truncate=False)


+---+--------+--------------------+----------------------+-------------------+-----------------------------+-----+-------+
|id |name    |last_modified_date  |new_last_modified_date|record_load_time   |record_type                  |age  |salary |
+---+--------+--------------------+----------------------+-------------------+-----------------------------+-----+-------+
|3  |Gabe    |03/08/2019  12:00:00|2019-08-03 12:00:00   |2019-11-29 09:57:54|#remove this record          |-21.0|-800.0 |
|1  |John    |01/08/2019  12:00:00|2019-08-01 12:00:00   |2019-11-29 09:57:54|#remove this record          |-30.0|-1000.0|
|2  |Peter   |02/08/2019  12:00:00|2019-08-02 12:00:00   |2019-11-29 09:57:54|#remove this record          |-35.0|-1500.0|
|3  |Gabe    |12/08/2019  12:00:00|2019-08-12 12:00:00   |2019-11-29 09:57:55|#keep this new updated record|21   |800    |
|5  |Anna    |05/08/2019  12:00:00|2019-08-05 12:00:00   |2019-11-29 09:57:55|#keep this new updated record|20   |1200   |
|6  |Patricia|12

In [24]:
current_data_df1.columns

['id',
 'name',
 'age',
 'salary',
 'last_modified_date',
 'new_last_modified_date',
 'record_load_time']

In [25]:
# finally merge delta with target current table
final_current_df = current_data_df1.union(final_update_df.select(current_data_df1.columns))
final_current_df.show(truncate=False)

+---+--------+-----+-------+--------------------+----------------------+-------------------+
|id |name    |age  |salary |last_modified_date  |new_last_modified_date|record_load_time   |
+---+--------+-----+-------+--------------------+----------------------+-------------------+
|1  |John    |30   |1000   |01/08/2019  12:00:00|2019-08-01 12:00:00   |2019-08-01 12:00:00|
|2  |Peter   |35   |1500   |02/08/2019  12:00:00|2019-08-02 12:00:00   |2019-08-02 12:00:00|
|3  |Gabe    |21   |800    |03/08/2019  12:00:00|2019-08-03 12:00:00   |2019-08-03 12:00:00|
|4  |Oscar   |29   |2000   |04/08/2019  12:00:00|2019-08-04 12:00:00   |2019-08-04 12:00:00|
|5  |Anna    |20   |1200   |05/08/2019  12:00:00|2019-08-05 12:00:00   |2019-08-05 12:00:00|
|3  |Gabe    |-21.0|-800.0 |03/08/2019  12:00:00|2019-08-03 12:00:00   |2019-11-29 09:57:54|
|1  |John    |-30.0|-1000.0|01/08/2019  12:00:00|2019-08-01 12:00:00   |2019-11-29 09:57:54|
|2  |Peter   |-35.0|-1500.0|02/08/2019  12:00:00|2019-08-02 12:00:00  

In [26]:
#final_current_df.filter("id <= 3").sort(col("id").asc()).show(truncate=False)
final_current_df.filter("id <= 3").sort(asc("id"), desc("record_load_time")).show(truncate=False)

+---+-----+-----+-------+--------------------+----------------------+-------------------+
|id |name |age  |salary |last_modified_date  |new_last_modified_date|record_load_time   |
+---+-----+-----+-------+--------------------+----------------------+-------------------+
|1  |John |43   |3000   |12/08/2019  12:00:00|2019-08-12 12:00:00   |2019-11-29 09:57:55|
|1  |John |-30.0|-1000.0|01/08/2019  12:00:00|2019-08-01 12:00:00   |2019-11-29 09:57:54|
|1  |John |30   |1000   |01/08/2019  12:00:00|2019-08-01 12:00:00   |2019-08-01 12:00:00|
|2  |Peter|35   |3500   |12/08/2019  12:00:00|2019-08-12 12:00:00   |2019-11-29 09:57:55|
|2  |Peter|-35.0|-1500.0|02/08/2019  12:00:00|2019-08-02 12:00:00   |2019-11-29 09:57:54|
|2  |Peter|35   |1500   |02/08/2019  12:00:00|2019-08-02 12:00:00   |2019-08-02 12:00:00|
|3  |Gabe |21   |800    |12/08/2019  12:00:00|2019-08-12 12:00:00   |2019-11-29 09:57:55|
|3  |Gabe |-21.0|-800.0 |03/08/2019  12:00:00|2019-08-03 12:00:00   |2019-11-29 09:57:54|
|3  |Gabe 

In [27]:
final_current_df.dtypes

[('id', 'string'),
 ('name', 'string'),
 ('age', 'string'),
 ('salary', 'string'),
 ('last_modified_date', 'string'),
 ('new_last_modified_date', 'timestamp'),
 ('record_load_time', 'timestamp')]

In [28]:
# identify the record to be delete between previous dataframe and new dataframe, use () with & operator to add multiple columns join condition
increamental_data_df2 = increamental_data_df1.withColumnRenamed("id", "new_id")
record_to_delete_df = current_data_df1.join(increamental_data_df2, (current_data_df1.id == increamental_data_df2.new_id) & (current_data_df1.name == increamental_data_df2.name),how='left').filter(col('new_id').isNull()).select(current_data_df1["*"],increamental_data_df2["new_id"])
record_to_delete_df.show()

+---+-----+---+------+--------------------+----------------------+-------------------+------+
| id| name|age|salary|  last_modified_date|new_last_modified_date|   record_load_time|new_id|
+---+-----+---+------+--------------------+----------------------+-------------------+------+
|  4|Oscar| 29|  2000|04/08/2019  12:00:00|   2019-08-04 12:00:00|2019-08-04 12:00:00|  null|
|  5| Anna| 20|  1200|05/08/2019  12:00:00|   2019-08-05 12:00:00|2019-08-05 12:00:00|  null|
+---+-----+---+------+--------------------+----------------------+-------------------+------+



In [29]:
# if records need to be deleted
import pyspark.sql.functions as F
# key figures need to be reverse
keyfigures = ['age', 'salary']
# column list except key figures
column_minus_keyfigures_list = [c for c in record_to_delete_df.columns if c not in keyfigures]

df_to_delete = record_to_delete_df.select( *column_minus_keyfigures_list , *[(col(col_name)* -1).alias(col_name) for col_name in keyfigures])
df_to_delete = df_to_delete.drop('new_id')
# reorder columns as per target table
df_to_delete.select([c for c in current_data_df1.columns]).show()

+---+-----+-----+-------+--------------------+----------------------+-------------------+
| id| name|  age| salary|  last_modified_date|new_last_modified_date|   record_load_time|
+---+-----+-----+-------+--------------------+----------------------+-------------------+
|  4|Oscar|-29.0|-2000.0|04/08/2019  12:00:00|   2019-08-04 12:00:00|2019-08-04 12:00:00|
|  5| Anna|-20.0|-1200.0|05/08/2019  12:00:00|   2019-08-05 12:00:00|2019-08-05 12:00:00|
+---+-----+-----+-------+--------------------+----------------------+-------------------+



In [30]:
# identify the record to be delete between previous dataframe and new dataframe based on multiple columns
df = current_data_df1.join(increamental_data_df2, (current_data_df1.id == increamental_data_df2.new_id) & (current_data_df1.name == increamental_data_df2.name ),'left').filter(col('new_id').isNull()).select(current_data_df1["*"],increamental_data_df2["new_id"])
col_list=["id","name","age"]
col_list=[c for c in df.columns]
df.select(col_list).show()


+---+-----+---+------+--------------------+----------------------+-------------------+------+
| id| name|age|salary|  last_modified_date|new_last_modified_date|   record_load_time|new_id|
+---+-----+---+------+--------------------+----------------------+-------------------+------+
|  4|Oscar| 29|  2000|04/08/2019  12:00:00|   2019-08-04 12:00:00|2019-08-04 12:00:00|  null|
|  5| Anna| 20|  1200|05/08/2019  12:00:00|   2019-08-05 12:00:00|2019-08-05 12:00:00|  null|
+---+-----+---+------+--------------------+----------------------+-------------------+------+



In [31]:
# A Transformer pipeline using SQLTransformer objects (or any other Transformer) is another approach which makes chaining easy.
from pyspark.ml.feature import SQLTransformer
from pyspark.ml import Pipeline, PipelineModel


df = spark.createDataFrame([
    (0, 1.0, 3.0),
    (2, 2.0, 5.0)
], ["id", "v1", "v2"])
sqlTrans = SQLTransformer(
    statement="SELECT *, (v1 + v2) AS v3, (v1 * v2) AS v4 FROM __THIS__")

sqlSelectExpr = SQLTransformer(statement="SELECT *, (id * 2) AS id2 FROM __THIS__")

pipeline = Pipeline(stages=[sqlTrans, sqlSelectExpr])
pipelineModel = pipeline.fit(df)
pipelineModel.transform(df).show()

+---+---+---+---+----+---+
| id| v1| v2| v3|  v4|id2|
+---+---+---+---+----+---+
|  0|1.0|3.0|4.0| 3.0|  0|
|  2|2.0|5.0|7.0|10.0|  4|
+---+---+---+---+----+---+



In [32]:

# prefix all columns of dataframe
df.printSchema()
def prefix_dataframe_columns(datafame , prefix):
    for col_name in datafame.columns:
        new_c = prefix + col_name 
        datafame = datafame.withColumnRenamed(col_name, new_c)
    return datafame
    
df1 = prefix_dataframe_columns(df,'prefix_')
df1.show()

root
 |-- id: long (nullable = true)
 |-- v1: double (nullable = true)
 |-- v2: double (nullable = true)

+---------+---------+---------+
|prefix_id|prefix_v1|prefix_v2|
+---------+---------+---------+
|        0|      1.0|      3.0|
|        2|      2.0|      5.0|
+---------+---------+---------+



In [33]:
# use if else to derive the new column as UDF
def return_age_bracket(age):
    if (age <= 12):
        return 'Under 12'
    elif (age >= 13 and age <= 19):
        return 'Between 13 and 19'
    elif (age > 19 and age < 65):
        return 'Between 19 and 65'
    elif (age >= 65):
        return 'Over 65'
    else: return 'N/A'

from pyspark.sql.functions import udf

maturity_udf = udf(return_age_bracket)
df = spark.createDataFrame([
    ('John', 1),
    ('David', 15),
    ('Raj', 31)
], ["name", "age"])
df = df.withColumn("maturity", maturity_udf(df.age))
df.show()

+-----+---+-----------------+
| name|age|         maturity|
+-----+---+-----------------+
| John|  1|         Under 12|
|David| 15|Between 13 and 19|
|  Raj| 31|Between 19 and 65|
+-----+---+-----------------+



In [34]:
# faster if else condition in pyspark
from pyspark.sql import functions as F
input_df = spark.createDataFrame([
    ('John', 1),
    ('David', 15),
    ('Raj', 31)
], ["name", "Age"])
input_df = input_df.withColumn("age_bracket", F.when(input_df.Age <= 12, 'Infant').when(( (input_df.Age >= 13) & (input_df.Age <= 19)), 'Adolescent').when(( (input_df.Age >= 19) & (input_df.Age < 65)), 'Adult').when(input_df.Age >= 65, 'Retired').otherwise('N/A'))
input_df.show()


+-----+---+-----------+
| name|Age|age_bracket|
+-----+---+-----------+
| John|  1|     Infant|
|David| 15| Adolescent|
|  Raj| 31|      Adult|
+-----+---+-----------+



In [35]:
# date timestamp management
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark.sql.functions as F
# convert string format to datetime format
from pyspark.sql.functions import unix_timestamp, from_unixtime
from pyspark.sql.functions import expr

file_schema = StructType([
  StructField("id", IntegerType(), False),
  StructField("name", StringType(), False),
  StructField("age", IntegerType(), False),
  StructField("salary", IntegerType(), False),
  StructField("last_modified_date", StringType(), False),
])

# existing data
test1 = spark.createDataFrame([(1, 'John',30,1000,'01/08/2019  12:00:00 AM'),
(2,'Peter',35,1500,'02/08/2019  12:00:00 AM'),
(3, 'Gabe',21,800,'03/08/2019  12:00:90 AM'),
(4,'Oscar',29,2000,'04/08/2019  12:00:40 AM'),
(5, 'Anna',20,1200,'invalid date/08/2019  12:00:00 AM')], file_schema)

test1 = test1.withColumn("new_last_modified_date_time",expr("to_timestamp(last_modified_date, 'dd/MM/yyyy  HH:mm:ss')"))
test1 = test1.withColumn("new_last_modified_date",expr("to_date(last_modified_date, 'dd/MM/yyyy  HH:mm:ss')"))

test1.show()
print(test1.dtypes)

+---+-----+---+------+--------------------+---------------------------+----------------------+
| id| name|age|salary|  last_modified_date|new_last_modified_date_time|new_last_modified_date|
+---+-----+---+------+--------------------+---------------------------+----------------------+
|  1| John| 30|  1000|01/08/2019  12:00...|        2019-08-01 12:00:00|            2019-08-01|
|  2|Peter| 35|  1500|02/08/2019  12:00...|        2019-08-02 12:00:00|            2019-08-02|
|  3| Gabe| 21|   800|03/08/2019  12:00...|                       null|                  null|
|  4|Oscar| 29|  2000|04/08/2019  12:00...|        2019-08-04 12:00:40|            2019-08-04|
|  5| Anna| 20|  1200|invalid date/08/2...|                       null|                  null|
+---+-----+---+------+--------------------+---------------------------+----------------------+

[('id', 'int'), ('name', 'string'), ('age', 'int'), ('salary', 'int'), ('last_modified_date', 'string'), ('new_last_modified_date_time', 'timest

In [36]:
# convert string of dd/MM/yyyy into date column by splitting string when invalid date need to be processed
from pyspark.sql.functions import expr
rfm_df1 = test1.withColumn("Order_date_new",expr("to_date(last_modified_date, 'dd/MM/yyyy')"))
rfm_df1 = rfm_df1.withColumn("Order_date_new2",expr("to_date(concat(substring(last_modified_date,7,4),substring(last_modified_date,4,2),substring(last_modified_date,1,2)), 'yyyyMMdd')"))
rfm_df1.printSchema()
rfm_df1.show()

root
 |-- id: integer (nullable = false)
 |-- name: string (nullable = false)
 |-- age: integer (nullable = false)
 |-- salary: integer (nullable = false)
 |-- last_modified_date: string (nullable = false)
 |-- new_last_modified_date_time: timestamp (nullable = true)
 |-- new_last_modified_date: date (nullable = true)
 |-- Order_date_new: date (nullable = true)
 |-- Order_date_new2: date (nullable = true)

+---+-----+---+------+--------------------+---------------------------+----------------------+--------------+---------------+
| id| name|age|salary|  last_modified_date|new_last_modified_date_time|new_last_modified_date|Order_date_new|Order_date_new2|
+---+-----+---+------+--------------------+---------------------------+----------------------+--------------+---------------+
|  1| John| 30|  1000|01/08/2019  12:00...|        2019-08-01 12:00:00|            2019-08-01|    2019-08-01|     2019-08-01|
|  2|Peter| 35|  1500|02/08/2019  12:00...|        2019-08-02 12:00:00|            201